In [1]:
!pip install scikit-learn torch pandas numpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
df = pd.read_csv('./dataset.csv')

In [3]:
num_columns = len(df.columns)
num_columns

1153

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import pandas as pd
from typing import List, Tuple

class ErrorCorrectionDataset:
    def __init__(self, df: pd.DataFrame, ber_pools: List[float], train_size: float = 0.8):
        self.df = df
        self.ber_pools = ber_pools
        self.train_size = train_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.original_cols = [c for c in df.columns if c.startswith("Original_")]
        self.corrupted_cols = [c for c in df.columns if c.startswith("Corrupted_")]

    def _split_tensor(self, sub_df: pd.DataFrame) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """Helper to split a dataframe into train/test torch tensors."""
        n_train = int(self.train_size * len(sub_df))
        sub_df = sub_df.sample(frac=1, random_state=None).reset_index(drop=True)

        X = sub_df[self.corrupted_cols].values.astype("float32") / 255.0
        y = sub_df[self.original_cols].values.astype("float32") / 255.0

        X_train, X_test = X[:n_train], X[n_train:]
        y_train, y_test = y[:n_train], y[n_train:]

        return (
            torch.tensor(X_train, dtype=torch.float32).to(self.device),
            torch.tensor(y_train, dtype=torch.float32).to(self.device),
            torch.tensor(X_test, dtype=torch.float32).to(self.device),
            torch.tensor(y_test, dtype=torch.float32).to(self.device),
        )

    def prepare_datasets(self) -> List[Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]]:
        """
        Returns a list of datasets (X_train, y_train, X_test, y_test) for each BER pool.
        """
        datasets = []
        for ber in self.ber_pools:
            sub_df = self.df[self.df["BER"] == ber]
            datasets.append(self._split_tensor(sub_df))
        return datasets

    def generalized_dataset(self) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Returns a single dataset (X_train, y_train, X_test, y_test) combining all BER pools.
        """
        return self._split_tensor(self.df)

In [7]:
ber_pools = [0.0, 0.2, 0.5, 0.8]
Dataset = ErrorCorrectionDataset(df, ber_pools, train_size=0.8)

In [8]:
error_datasets = Dataset.prepare_datasets()
generalized_dataset = Dataset.generalized_dataset()

In [9]:
class ErrorCorrector(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, n_features),
            nn.Sigmoid()
        )
    def forward(self, x): return self.net(x)

In [10]:
n_features = num_columns // 2
model = ErrorCorrector(n_features).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
X_train_00 = error_datasets[0][0]
y_train_00 = error_datasets[0][1]

for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_00)
    loss = criterion(outputs, y_train_00)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.693780
Epoch 2, Loss=0.692336
Epoch 3, Loss=0.691544
Epoch 4, Loss=0.690953
Epoch 5, Loss=0.690507
Epoch 6, Loss=0.690222
Epoch 7, Loss=0.689954
Epoch 8, Loss=0.689662
Epoch 9, Loss=0.689352
Epoch 10, Loss=0.689012
Epoch 11, Loss=0.688621
Epoch 12, Loss=0.688208
Epoch 13, Loss=0.687774
Epoch 14, Loss=0.687306
Epoch 15, Loss=0.686812
Epoch 16, Loss=0.686291
Epoch 17, Loss=0.685733
Epoch 18, Loss=0.685134
Epoch 19, Loss=0.684524
Epoch 20, Loss=0.683952
Epoch 21, Loss=0.683435
Epoch 22, Loss=0.682609
Epoch 23, Loss=0.681861
Epoch 24, Loss=0.681275
Epoch 25, Loss=0.680410
Epoch 26, Loss=0.679645
Epoch 27, Loss=0.678911
Epoch 28, Loss=0.678010
Epoch 29, Loss=0.677182
Epoch 30, Loss=0.676348
Epoch 31, Loss=0.675421
Epoch 32, Loss=0.674601
Epoch 33, Loss=0.674092
Epoch 34, Loss=0.673577
Epoch 35, Loss=0.672383
Epoch 36, Loss=0.671841
Epoch 37, Loss=0.671187
Epoch 38, Loss=0.670680
Epoch 39, Loss=0.669954
Epoch 40, Loss=0.669123
Epoch 41, Loss=0.668295
Epoch 42, Loss=0.668033
E

In [12]:
X_train_02 = error_datasets[1][0]
y_train_02 = error_datasets[1][1]

for epoch in range(60):
    optimizer.zero_grad()
    outputs = model(X_train_02)
    loss = criterion(outputs, y_train_02)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.659095
Epoch 2, Loss=0.659102
Epoch 3, Loss=0.658696
Epoch 4, Loss=0.658377
Epoch 5, Loss=0.658324
Epoch 6, Loss=0.657725
Epoch 7, Loss=0.657372
Epoch 8, Loss=0.657420
Epoch 9, Loss=0.656842
Epoch 10, Loss=0.656712
Epoch 11, Loss=0.656581
Epoch 12, Loss=0.656127
Epoch 13, Loss=0.656123
Epoch 14, Loss=0.655745
Epoch 15, Loss=0.655605
Epoch 16, Loss=0.655392
Epoch 17, Loss=0.655119
Epoch 18, Loss=0.655005
Epoch 19, Loss=0.654697
Epoch 20, Loss=0.654590
Epoch 21, Loss=0.654306
Epoch 22, Loss=0.654172
Epoch 23, Loss=0.653945
Epoch 24, Loss=0.653755
Epoch 25, Loss=0.653577
Epoch 26, Loss=0.653323
Epoch 27, Loss=0.653186
Epoch 28, Loss=0.652943
Epoch 29, Loss=0.652797
Epoch 30, Loss=0.652611
Epoch 31, Loss=0.652422
Epoch 32, Loss=0.652240
Epoch 33, Loss=0.652032
Epoch 34, Loss=0.651858
Epoch 35, Loss=0.651666
Epoch 36, Loss=0.651510
Epoch 37, Loss=0.651346
Epoch 38, Loss=0.651180
Epoch 39, Loss=0.651041
Epoch 40, Loss=0.650888
Epoch 41, Loss=0.650742
Epoch 42, Loss=0.650579
E

In [13]:
X_train_05 = error_datasets[2][0]
y_train_05 = error_datasets[2][1]

for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(X_train_05)
    loss = criterion(outputs, y_train_05)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.677091
Epoch 2, Loss=0.677451
Epoch 3, Loss=0.677787
Epoch 4, Loss=0.677483
Epoch 5, Loss=0.676791
Epoch 6, Loss=0.676596
Epoch 7, Loss=0.676641
Epoch 8, Loss=0.676182
Epoch 9, Loss=0.675914
Epoch 10, Loss=0.675907
Epoch 11, Loss=0.675466
Epoch 12, Loss=0.675289
Epoch 13, Loss=0.675119
Epoch 14, Loss=0.674769
Epoch 15, Loss=0.674645
Epoch 16, Loss=0.674327
Epoch 17, Loss=0.674165
Epoch 18, Loss=0.673893
Epoch 19, Loss=0.673706
Epoch 20, Loss=0.673477


In [14]:
X_train_08 = error_datasets[3][0]
y_train_08 = error_datasets[3][1]

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train_08)
    loss = criterion(outputs, y_train_08)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.690970
Epoch 2, Loss=0.690746
Epoch 3, Loss=0.690396
Epoch 4, Loss=0.690054
Epoch 5, Loss=0.689600
Epoch 6, Loss=0.689132
Epoch 7, Loss=0.688616
Epoch 8, Loss=0.688094
Epoch 9, Loss=0.687588
Epoch 10, Loss=0.687144


In [15]:
X_train = generalized_dataset[0]
y_train = generalized_dataset[1]

for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=0.664328
Epoch 2, Loss=0.664662
Epoch 3, Loss=0.664613
Epoch 4, Loss=0.664188
Epoch 5, Loss=0.663560
Epoch 6, Loss=0.662773
Epoch 7, Loss=0.661944
Epoch 8, Loss=0.661126
Epoch 9, Loss=0.660486
Epoch 10, Loss=0.660104
Epoch 11, Loss=0.659872
Epoch 12, Loss=0.659651
Epoch 13, Loss=0.659421
Epoch 14, Loss=0.659322
Epoch 15, Loss=0.659249
Epoch 16, Loss=0.659121
Epoch 17, Loss=0.659037
Epoch 18, Loss=0.659040
Epoch 19, Loss=0.659243
Epoch 20, Loss=0.660288


In [16]:
X_test = generalized_dataset[2]
y_test = generalized_dataset[3]

model.eval()
with torch.no_grad():
    # ---- Test metrics ----
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)

    test_preds = (test_outputs * 255).round().int().cpu().numpy()
    test_targets = (y_test * 255).round().int().cpu().numpy()
    test_acc = (test_preds == test_targets).mean()

    print(f"Test  Loss = {test_loss.item():.6f}, Test Byte Accuracy = {test_acc*100:.6f}%")

Test  Loss = 0.662494, Test Byte Accuracy = 0.500357%


In [17]:
model.eval()
with torch.no_grad():
    # ---- Train metrics ----
    train_outputs = model(X_train)
    train_loss = criterion(train_outputs, y_train)

    train_preds = (train_outputs * 255).round().int().cpu().numpy()
    train_targets = (y_train * 255).round().int().cpu().numpy()
    train_acc = (train_preds == train_targets).mean()

    print(f"Train Loss = {train_loss.item():.6f}, Train Byte Accuracy = {train_acc*100:.6f}%")

Train Loss = 0.662961, Train Byte Accuracy = 0.513877%


In [ ]:
import os

save_path = "./error_corrector_updated_dataset.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

Model saved to ./error_correction/error_corrector_updated_dataset.pth
